## Idea Trabajo Práctico Final: Predicción de Resultado de Partidos de Tenis 
El objetivo de la red neuronal a implementar es poder predecir los resultados de cierta temporada de tenis actual considerando el historial de partidos de temporadas pasadas, como así también el historial individual entre los jugadores de cada partido. 
### Dataset de Referencia
El dataset de referencia será el historial de partidos de la Asociación de Tenistas Profesionales (ATP) brindado en su base de datos. Este no sólo incluye quien ha ganado cada partido, sino el puntaje y los porcentajes de las características relevantes de cada jugada en particular ('primeros servicios', 'segundos servicios', 'aces', 'porcentaje de devoluciones', 'doble faltas', etc.).
Por lo tanto, el set de entrenamiento de la red consistirá de un vector de características de entrada (X) representando las características de los jugadores involucrados y del partido, y un vector de salida correspondiente que determina que jugador gana. 
A continuación se realiza el parseo de datos:


In [2]:
import numpy as np
import pandas as pd
import re
import numpy


from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

#test = pd.read_csv('data/match_stats_2017_UNINDEXED.csv')
#pd_atp_matches_2017.loc[[0,1,2,3,4,5,9,1000,1001,1002,1003,1004]]
#pd.set_option('display.max_rows', 5000)
#pd_atp_matches_2017
#pd_atp_matches_2017.iloc[1]

In [65]:
my_data_stats = pd.read_csv('data/match_stats_2017_UNINDEXED.csv')

my_data_stats

,ms_tourney_order,ms_match_id,ms_match_stats_url_suffix,ms_match_time,ms_match_duration,ms_winner_aces,ms_winner_double_faults,ms_winner_first_serves_in,ms_winner_first_serves_total,ms_winner_first_serve_points_won,...,ms_loser_second_serve_return_won,ms_loser_second_serve_return_total,ms_loser_break_points_converted,ms_loser_break_points_return_total,ms_loser_service_games_played,ms_loser_return_games_played,ms_loser_return_points_won,ms_loser_return_points_total,ms_loser_total_points_won,ms_loser_total_points_total
0,0,2017-339-r975-n409,/en/scores/2017/339/MS004/match-stats,02:21:00,141,23,3,62,97,50,...,19,35,1,7,14,15,31,97,84,181
1,0,2017-339-d875-n552,/en/scores/2017/339/MS001/match-stats,01:49:00,109,7,2,52,77,41,...,13,25,2,7,12,13,24,77,69,146
2,0,2017-339-d875-r975,/en/scores/2017/339/MS002/match-stats,01:28:00,88,4,1,36,58,27,...,4,22,0,0,10,10,13,58,53,119
3,0,2017-339-n552-w367,/en/scores/2017/339/MS003/match-stats,01:42:00,102,1,1,56,77,37,...,7,21,1,5,10,11,26,77,63,138
4,0,2017-339-d875-tb69,/en/scores/2017/339/MS005/match-stats,02:05:00,125,3,3,52,94,42,...,19,42,1,14,14,14,29,94,82,176
5,0,2017-339-w367-e831,/en/scores/2017/339/MS007/match-stats,02:36:00,156,11,3,67,119,47,...,24,52,2,13,16,16,44,119,107,216
6,0,2017-339-n552-tc61,/en/scores/2017/339/MS006/match-stats,01:02:00,62,3,0,19,34,18,...,5,15,0,0,7,7,6,34,26,81
7,0,2017-339-r975-sm37,/en/scores/2017/339/MS008/match-stats,01:09:00,69,12,1,40,53,30,...,6,13,1,3,8,9,16,53,39,97
8,0,2017-339-tb69-g940,/en/scores/2017/339/MS010/match-stats,01:29:00,89,11,1,44,65,36,...,6,21,0,4,10,11,14,65,57,123
9,0,2017-339-w367-t840,/en/scores/2017/339/MS015/match-stats,01:30:00,90,7,2,41,64,33,...,8,23,1,1,11,11,16,64,63,135


In [4]:
pd_atp_matches_2017 = pd.read_csv('tennis_atp/atp_matches_2017.csv')
my_data_stats = pd.read_csv('data/match_stats_2017_UNINDEXED.csv')

#select what do I want from the stats dataset
mycolumns=[
           'ms_winner_aces','ms_loser_aces','ms_winner_double_faults','ms_loser_double_faults',
           'ms_winner_first_serves_total','ms_winner_first_serve_points_won','ms_winner_first_serves_in',
           'ms_loser_first_serves_total','ms_loser_first_serve_points_won','ms_loser_first_serves_in',
           'ms_winner_return_points_won','ms_winner_return_points_total',
           'ms_loser_return_points_won','ms_loser_return_points_total',
           'ms_winner_total_points_won','ms_winner_total_points_total',
            'ms_winner_second_serve_points_total','ms_loser_second_serve_points_total']
new_columns=[
             'w_ace','l_ace','w_df','l_df',
             'w_svpt','w_1stWon','w_1stIn',
             'l_svpt','l_1stWon','l_1stIn',   
             'w_ret_w','w_ret_tot',
             'l_ret_w','l_ret_tot',
             'w_pts_w','total_points',
            'w_2ndTot','l_2ndTot']

my_data_stats=my_data_stats.filter(mycolumns, axis=1)
my_data_stats.columns = new_columns

#select what I want from the other dataset
labels=['tourney_id','tourney_name','round','winner_id','winner_name','loser_id','loser_name','score',
        'winner_rank','winner_rank_points','loser_rank','loser_rank_points',
        'w_ace','l_ace','w_df','l_df',
        'w_svpt','w_1stWon','w_1stIn',
        'l_svpt','l_1stWon','l_1stIn',
        'w_2ndWon','l_2ndWon',
        'w_bpSaved','w_bpFaced',
        'l_bpSaved','l_bpFaced']
pd_atp_matches_2017=pd_atp_matches_2017.filter(labels, axis=1)

labels_match=['w_ace','l_ace','w_df','l_df',
                'w_svpt','w_1stWon','w_1stIn',
                'l_svpt','l_1stWon','l_1stIn']
pd_atp_matches_2017=pd.merge(pd_atp_matches_2017,my_data_stats,on=labels_match).reset_index(drop=True)
pd_atp_matches_2017=pd_atp_matches_2017.drop_duplicates(keep='first').reset_index(drop=True)

WRP1=pd.read_csv('WRP1.csv')
pd_atp_matches_2017['WRP1'] = WRP1['WRP1']

WRP2=pd.read_csv('WRP2.csv')
pd_atp_matches_2017['WRP2'] = WRP2['WRP2']

WRP=pd.read_csv('WRP.csv')
pd_atp_matches_2017['WRP'] = WRP['WRP']

TMW=pd.read_csv('TMW.csv')
pd_atp_matches_2017['TMW'] = TMW['TMW']


#calculate number of games
def findOccurrences(s, ch):
    return [i for i, letter in enumerate(s) if letter == ch]

def change_char(s, p, r):
    return s[:p]+r+s[p+1:]

pd_atp_matches_2017=pd_atp_matches_2017.assign(TotGames=0)
for i in range(0,len(pd_atp_matches_2017.index)):
    str1=pd_atp_matches_2017.loc[i, 'score']
    
    start=findOccurrences(str1, '(')
    end=findOccurrences(str1, ')')

    for k in range(0,len(start)):
        for l in range(0,end[k]-start[k]+1):
            str1=change_char(str1,start[k]+l,' ')

    number_of_games=0
    for j in range (0,len(list(filter(str.isdigit, str1)))):
        number_of_games=number_of_games+int(list(filter(str.isdigit, str1))[j])
    pd_atp_matches_2017.loc[i, 'TotGames'] = number_of_games

    
    
def SetNan(pandas,column):
    list = pandas.index[(pandas[column].isin([0])) == True].tolist()
    for i in range(0,len(list)):
        pandas.loc[list[i],column]=np.nan
        
SetNan(pd_atp_matches_2017,'w_svpt')
SetNan(pd_atp_matches_2017,'l_svpt')
SetNan(pd_atp_matches_2017,'w_1stIn')
SetNan(pd_atp_matches_2017,'l_1stIn')
SetNan(pd_atp_matches_2017,'w_2ndTot')
SetNan(pd_atp_matches_2017,'l_2ndTot')

Debajo se muestra un ejemplo del dataset scores obtenido:

In [81]:
display(pd_atp_matches_2017.loc[0])

tourney_id                  2017-M020
tourney_name                 Brisbane
round                               F
winner_id                      105777
winner_name           Grigor Dimitrov
loser_id                       105453
loser_name              Kei Nishikori
score                     6-2 2-6 6-3
winner_rank                        17
winner_rank_points               2035
loser_rank                          5
loser_rank_points                4905
w_ace                               7
l_ace                               4
w_df                                2
l_df                                0
w_svpt                             77
w_1stWon                           41
w_1stIn                            52
l_svpt                             69
l_1stWon                           36
l_1stIn                            49
w_2ndWon                           12
l_2ndWon                            9
w_ret_w                            24
w_ret_tot                          69
l_ret_w     

## Extracción de Features 
A continuación se describiran la extracción de features de cada partido. 

A continuación se busca la lista de todos los jugadores presentes.

In [9]:
all_winner_players=pd_atp_matches_2017.filter(['winner_id','winner_name'], axis=1)
all_winner_players.rename(columns={'winner_id': 'ID','winner_name': 'NAME'}, inplace=True)

all_loser_players=pd_atp_matches_2017.filter(['loser_id','loser_name'], axis=1)
all_loser_players.rename(columns={'loser_id': 'ID','loser_name': 'NAME'}, inplace=True)

#print(all_winner_players)
#print(all_loser_players)

all_players=all_winner_players.append(all_loser_players, ignore_index=True).drop_duplicates()

all_players=all_players.sort_values(by=['ID']).reset_index(drop=True);
#players_list=all_players.ID.unique()
#players_list.sort()
#players_list
#all_players

Luego, se determina el feature que indica cual jugador tuvo mejor historial en los partidos anteriores.

In [112]:
#this function returns WRP of a player against a certain adversary

def getMatchesWonPer(myopponent,adversary):
    exp1=pd_atp_matches_2017['winner_id'].isin([myopponent])
    exp2=pd_atp_matches_2017['loser_id'].isin([adversary])
    exp3=pd_atp_matches_2017['loser_id'].isin([myopponent])
    exp4=pd_atp_matches_2017['winner_id'].isin([adversary])

    matches_won=len(pd_atp_matches_2017.loc[(exp1&exp2)].index)
    matches_lost=len(pd_atp_matches_2017.loc[(exp3&exp4)].index)
    matchesWonPer = matches_won/(matches_won+matches_lost)
    
    return matchesWonPer

pd_atp_matches_2017=pd_atp_matches_2017.assign(TMW=0)
for i in range(0,len(pd_atp_matches_2017.index)):
    TMW1=getMatchesWonPer(pd_atp_matches_2017.loc[i, 'winner_id'],pd_atp_matches_2017.loc[i, 'loser_id'])
    pd_atp_matches_2017.loc[i, 'TMW'] = TMW1 - (1-TMW1)
    
pd_atp_matches_2017['TMW'].to_csv('TMW.csv',header='false',index='false')

Se buscan los oponentes en común y se calcula el promedio del porcentaje de puntos devueltos ganados.

In [10]:
#this function returns WRP of a player against a certain adversary

def getWRPi(myopponent,adversary):
    exp1=pd_atp_matches_2017['winner_id'].isin([myopponent])
    exp2=pd_atp_matches_2017['loser_id'].isin([adversary])
    exp3=pd_atp_matches_2017['loser_id'].isin([myopponent])
    exp4=pd_atp_matches_2017['winner_id'].isin([adversary])

    matches_won=pd_atp_matches_2017.loc[(exp1&exp2)]
    matches_won=matches_won.filter(['tourney_name','winner_id','winner_name','loser_id','loser_name','w_ret_w','w_ret_tot','l_ret_w','l_ret_tot'], axis=1)

    matches_lost=pd_atp_matches_2017.loc[(exp3&exp4)]
    matches_lost=matches_lost.filter(['tourney_name','winner_id','winner_name','loser_id','loser_name','w_ret_w','w_ret_tot','l_ret_w','l_ret_tot'], axis=1)

    #calculate WRP (winning return percentage) of all the matches against the adversary
    matches_won['WRP'] = matches_won['w_ret_w']/matches_won['w_ret_tot']
    matches_lost['WRP'] = matches_lost['l_ret_w']/matches_lost['l_ret_tot']
    matches_won=matches_won.append(matches_lost)
    matches_won
    return matches_won['WRP'].mean()


def getWRP(id1,id2):

    #id1 = 105223 #delpo
    #id2 = 105992 #Ryan harrison

    #id1 = 104229 #nadal
    #id2 = 105032 #groth
    array=[id1,id2]
    locate_1=all_players.loc[(all_players['ID'].isin([array[0]]))]
    locate_2=all_players.loc[(all_players['ID'].isin([array[1]]))]

    #print('El jugador #1 es: ' + locate_1.iloc[0,1])
    #print('El jugador #2 es: ' + locate_2.iloc[0,1])

    #print('En el 2017 se enfrentaron a: ' )

    opponents_p1_1=pd_atp_matches_2017.loc[(pd_atp_matches_2017['winner_id'].isin([array[0]]))]
    opponents_p1_1=opponents_p1_1[['loser_id','loser_name']]
    opponents_p1_1.rename(columns={'loser_id': 'ID','loser_name': 'NAME'}, inplace=True)
    opponents_p1_2=pd_atp_matches_2017.loc[(pd_atp_matches_2017['loser_id'].isin([array[0]]))]
    opponents_p1_2=opponents_p1_2[['winner_id','winner_name']]
    opponents_p1_2.rename(columns={'winner_id': 'ID','winner_name': 'NAME'}, inplace=True)
    opponents_p1=opponents_p1_1.append(opponents_p1_2, ignore_index=True).drop_duplicates()

    opponents_p2_1=pd_atp_matches_2017.loc[(pd_atp_matches_2017['winner_id'].isin([array[1]]))]
    opponents_p2_1=opponents_p2_1[['loser_id','loser_name']]
    opponents_p2_1.rename(columns={'loser_id': 'ID','loser_name': 'NAME'}, inplace=True)
    opponents_p2_2=pd_atp_matches_2017.loc[(pd_atp_matches_2017['loser_id'].isin([array[1]]))]
    opponents_p2_2=opponents_p2_2[['winner_id','winner_name']]
    opponents_p2_2.rename(columns={'winner_id': 'ID','winner_name': 'NAME'}, inplace=True)
    opponents_p2=opponents_p2_1.append(opponents_p2_2, ignore_index=True).drop_duplicates()

    common_opponents = pd.merge(opponents_p1, opponents_p2, how='inner', on=['ID']).filter(['ID','NAME_x'], axis=1)
    common_opponents=common_opponents.assign(WRP1=0,WRP2=0)

    for i in range(0,len(common_opponents.index)):
        common_opponents.loc[i, 'WRP1'] = getWRPi(id1,common_opponents.loc[i, 'ID'])
        common_opponents.loc[i, 'WRP2'] = getWRPi(id2,common_opponents.loc[i, 'ID'])

    WRP1=common_opponents['WRP1'].mean()
    WRP2=common_opponents['WRP2'].mean()
    #WRP=WRP1-WRP2
    #print(WRP)

    #display_side_by_side(opponents_p1,opponents_p2,common_opponents)
    return(WRP1,WRP2)

##Run to get WRP

pd_atp_matches_2017=pd_atp_matches_2017.assign(WRP=0)

for i in range(0,len(pd_atp_matches_2017.index)):
    WRP1,WRP2 = getWRP(pd_atp_matches_2017.loc[i, 'winner_id'],pd_atp_matches_2017.loc[i, 'loser_id'])
    pd_atp_matches_2017.loc[i, 'WRP1'] = WRP1
    pd_atp_matches_2017.loc[i, 'WRP2'] = WRP2
    pd_atp_matches_2017.loc[i, 'WRP'] = WRP1-WRP2
    print('Processing match #' + str(i) + '\n')
    
pd_atp_matches_2017['WRP1'].to_csv('WRP1.csv',header='false',index='false')
pd_atp_matches_2017['WRP2'].to_csv('WRP2.csv',header='false',index='false')
pd_atp_matches_2017['WRP'].to_csv('WRP.csv',header='false',index='false')

Processing match #0

Processing match #1

Processing match #2

Processing match #3

Processing match #4

Processing match #5

Processing match #6

Processing match #7

Processing match #8

Processing match #9

Processing match #10

Processing match #11

Processing match #12

Processing match #13

Processing match #14

Processing match #15

Processing match #16

Processing match #17

Processing match #18

Processing match #19

Processing match #20

Processing match #21

Processing match #22

Processing match #23

Processing match #24

Processing match #25

Processing match #26

Processing match #27

Processing match #28

Processing match #29

Processing match #30

Processing match #31

Processing match #32

Processing match #33

Processing match #34

Processing match #35

Processing match #36

Processing match #37

Processing match #38

Processing match #39

Processing match #40

Processing match #41

Processing match #42

Processing match #43

Processing match #44

Processing match #45

Processing match #362

Processing match #363

Processing match #364

Processing match #365

Processing match #366

Processing match #367

Processing match #368

Processing match #369

Processing match #370

Processing match #371

Processing match #372

Processing match #373

Processing match #374

Processing match #375

Processing match #376

Processing match #377

Processing match #378

Processing match #379

Processing match #380

Processing match #381

Processing match #382

Processing match #383

Processing match #384

Processing match #385

Processing match #386

Processing match #387

Processing match #388

Processing match #389

Processing match #390

Processing match #391

Processing match #392

Processing match #393

Processing match #394

Processing match #395

Processing match #396

Processing match #397

Processing match #398

Processing match #399

Processing match #400

Processing match #401

Processing match #402

Processing match #403

Processing match #404

Processing 

Processing match #720

Processing match #721

Processing match #722

Processing match #723

Processing match #724

Processing match #725

Processing match #726

Processing match #727

Processing match #728

Processing match #729

Processing match #730

Processing match #731

Processing match #732

Processing match #733

Processing match #734

Processing match #735

Processing match #736

Processing match #737

Processing match #738

Processing match #739

Processing match #740

Processing match #741

Processing match #742

Processing match #743

Processing match #744

Processing match #745

Processing match #746

Processing match #747

Processing match #748

Processing match #749

Processing match #750

Processing match #751

Processing match #752

Processing match #753

Processing match #754

Processing match #755

Processing match #756

Processing match #757

Processing match #758

Processing match #759

Processing match #760

Processing match #761

Processing match #762

Processing 

Processing match #1074

Processing match #1075

Processing match #1076

Processing match #1077

Processing match #1078

Processing match #1079

Processing match #1080

Processing match #1081

Processing match #1082

Processing match #1083

Processing match #1084

Processing match #1085

Processing match #1086

Processing match #1087

Processing match #1088

Processing match #1089

Processing match #1090

Processing match #1091

Processing match #1092

Processing match #1093

Processing match #1094

Processing match #1095

Processing match #1096

Processing match #1097

Processing match #1098

Processing match #1099

Processing match #1100

Processing match #1101

Processing match #1102

Processing match #1103

Processing match #1104

Processing match #1105

Processing match #1106

Processing match #1107

Processing match #1108

Processing match #1109

Processing match #1110

Processing match #1111

Processing match #1112

Processing match #1113

Processing match #1114

Processing match

Processing match #1416

Processing match #1417

Processing match #1418

Processing match #1419

Processing match #1420

Processing match #1421

Processing match #1422

Processing match #1423

Processing match #1424

Processing match #1425

Processing match #1426

Processing match #1427

Processing match #1428

Processing match #1429

Processing match #1430

Processing match #1431

Processing match #1432

Processing match #1433

Processing match #1434

Processing match #1435

Processing match #1436

Processing match #1437

Processing match #1438

Processing match #1439

Processing match #1440

Processing match #1441

Processing match #1442

Processing match #1443

Processing match #1444

Processing match #1445

Processing match #1446

Processing match #1447

Processing match #1448

Processing match #1449

Processing match #1450

Processing match #1451

Processing match #1452

Processing match #1453

Processing match #1454

Processing match #1455

Processing match #1456

Processing match

Processing match #1758

Processing match #1759

Processing match #1760

Processing match #1761

Processing match #1762

Processing match #1763

Processing match #1764

Processing match #1765

Processing match #1766

Processing match #1767

Processing match #1768

Processing match #1769

Processing match #1770

Processing match #1771

Processing match #1772

Processing match #1773

Processing match #1774

Processing match #1775

Processing match #1776

Processing match #1777

Processing match #1778

Processing match #1779

Processing match #1780

Processing match #1781

Processing match #1782

Processing match #1783

Processing match #1784

Processing match #1785

Processing match #1786

Processing match #1787

Processing match #1788

Processing match #1789

Processing match #1790

Processing match #1791

Processing match #1792

Processing match #1793

Processing match #1794

Processing match #1795

Processing match #1796

Processing match #1797

Processing match #1798

Processing match

Processing match #2100

Processing match #2101

Processing match #2102

Processing match #2103

Processing match #2104

Processing match #2105

Processing match #2106

Processing match #2107

Processing match #2108

Processing match #2109

Processing match #2110

Processing match #2111

Processing match #2112

Processing match #2113

Processing match #2114

Processing match #2115

Processing match #2116

Processing match #2117

Processing match #2118

Processing match #2119

Processing match #2120

Processing match #2121

Processing match #2122

Processing match #2123

Processing match #2124

Processing match #2125

Processing match #2126

Processing match #2127

Processing match #2128

Processing match #2129

Processing match #2130

Processing match #2131

Processing match #2132

Processing match #2133

Processing match #2134

Processing match #2135

Processing match #2136

Processing match #2137

Processing match #2138

Processing match #2139

Processing match #2140

Processing match

Processing match #2442

Processing match #2443

Processing match #2444

Processing match #2445

Processing match #2446

Processing match #2447

Processing match #2448

Processing match #2449

Processing match #2450

Processing match #2451

Processing match #2452

Processing match #2453

Processing match #2454

Processing match #2455

Processing match #2456

Processing match #2457

Processing match #2458

Processing match #2459

Processing match #2460

Processing match #2461

Processing match #2462

Processing match #2463

Processing match #2464

Processing match #2465

Processing match #2466

Processing match #2467

Processing match #2468

Processing match #2469

Processing match #2470

Processing match #2471

Processing match #2472

Processing match #2473

Processing match #2474

Processing match #2475

Processing match #2476

Processing match #2477

Processing match #2478

Processing match #2479

Processing match #2480

Processing match #2481

Processing match #2482

Processing match

### Features para el entrenamiento

Los datos luego serán seleccionados partido a partido, a partir del dataset obtenido. Se realiza la siguiente división:

| Categoría  |  Detalle | 
|---|---|
| ID1 |  ID Jugador 1 |
| ID2 |  ID Jugador 2 |
| RANK |  Ranking ATP |
| POINTS  |  Puntos ATP | 
| FS | Porcentaje  de primeros servicios |
| W1SP | Porcentaje de primeros servicios ganados |
| W2SP | Porcentaje de segundos servicios ganados |
| WSP | Porcentaje de servicios ganados globales |
| WRP | Porcentaje de devolución ganada |
| TPW | Porcentaje de puntos totales ganados |
| TMW | Porcentaje de todos los partidos ganados |
| ACES | Promedio de aces por game |
| DF | Promedio de doble faltas por game |
| UE | Promedio de errores no forzados por game |
| WIS | Promedio de winners por game |
| BP | Porcentaje de break points ganados |
| NA | Porcentaje de puntos en la red ganados |
| A1S | Promedio de velocidad primer servicio |
| A2S | Promedio de velocidad de segundo servicio |
| FATIGUE | Fatiga por partidos jugados en 3 dias anteriores |
| RETIRED | Indica si es el primer partido luego de un retiro |
| DIRECT | Ventaja H2H entre los jugadores |

Cada partido tiene estas características que suelen indicar las diferencias entre ambos jugadores. Por lo que por ejemplo, para ATPRank, si RANK1 es el ranking del jugador 1 y RANK2 es el ranking del jugador 2, RANK = RANK1 - RANK2.
Para el entrenamiento, cada partido tendrá como salida un vector que inidque que jugador gana o que jugador pierde. Es decir, 1 si el jugador ganó o 0 si el jugador perdió.

Finalmente, el dataset será dividido en tres partes:
- Set de entrenamiento (2008 - 2014)
- Set de validación  (2015 - 2016)
- Set de prueba (2017-2018)


In [31]:
results = pd_atp_matches_2017.filter(['winner_name','loser_name'], axis=1)

results['RANK'] = pd_atp_matches_2017['winner_rank'] - pd_atp_matches_2017['loser_rank']
results['POINTS'] = pd_atp_matches_2017['winner_rank_points'] - pd_atp_matches_2017['loser_rank_points']


FS1 = pd_atp_matches_2017['w_1stIn'].div(pd_atp_matches_2017['w_svpt']) 
FS2 =  pd_atp_matches_2017['l_1stIn'].div(pd_atp_matches_2017['l_svpt'])
results['FS'] = FS1 - FS2

W1SP1 = pd_atp_matches_2017['w_1stWon'].div(pd_atp_matches_2017['w_1stIn'])
W1SP2 =  pd_atp_matches_2017['l_1stWon'].div(pd_atp_matches_2017['l_1stIn'])
results['W1SP'] = W1SP1 - W1SP2

W2SP1 = pd_atp_matches_2017['w_2ndWon'].div(pd_atp_matches_2017['w_2ndTot'])
W2SP2 =  pd_atp_matches_2017['l_2ndWon'].div(pd_atp_matches_2017['l_2ndTot'])
results['W2SP'] = W2SP1 - W2SP2

WSP1 = W1SP1 * FS1 + W2SP1 * (1-FS1)
WSP2 = W1SP2 * FS2 + W2SP2 * (1-FS2)
results['WSP'] = WSP1 - WSP2

WRP1=pd_atp_matches_2017['WRP1']
WRP2=pd_atp_matches_2017['WRP2']
results['WRP'] = pd_atp_matches_2017['WRP']

TPW1 = pd_atp_matches_2017['w_pts_w']/pd_atp_matches_2017['total_points']
TPW2 = (pd_atp_matches_2017['total_points']-pd_atp_matches_2017['w_pts_w'])/pd_atp_matches_2017['total_points']
results['TPW'] = TPW1 - (1-TPW1)

results['TMW'] = pd_atp_matches_2017['TMW']

results['ACES'] = (pd_atp_matches_2017['w_ace']-pd_atp_matches_2017['l_ace'])/pd_atp_matches_2017['TotGames']
results['DF'] = (pd_atp_matches_2017['w_df']-pd_atp_matches_2017['l_df'])/pd_atp_matches_2017['TotGames']

BP1 = pd_atp_matches_2017['w_bpSaved']/pd_atp_matches_2017['w_bpFaced']
BP2 = pd_atp_matches_2017['l_bpSaved']/pd_atp_matches_2017['l_bpFaced']
results['BP'] = BP1-BP2

COMPLETE1 = WSP1 * WRP1
COMPLETE2 = WSP2 * WRP2
results['COMPLETE'] = COMPLETE1-COMPLETE2

SERVEADV1 = WSP1 * WRP1
SERVEADV2 = WSP2 * WRP2
results['SERVEADV'] = COMPLETE1-COMPLETE2

#results.rename(columns{'winner_id': 'ID1', 'loser_id': 'ID2'}, inplace=True)

results=results.assign(winner=1)

results.loc[[0,1,2,3,4,5,9,1000,1001,1002,1003,1004]]


,winner_name,loser_name,RANK,POINTS,FS,W1SP,W2SP,WSP,WRP,TPW,TMW,ACES,DF,BP,COMPLETE,SERVEADV,winner
0,Grigor Dimitrov,Kei Nishikori,12.0,-2870.0,-0.0348203,0.0537677,0.030000,0.0361378,0.012796,0.054795,1.000000,0.12,0.08,0.314286,0.0228336,0.0228336,1
1,Grigor Dimitrov,Milos Raonic,14.0,-3415.0,0.161673,-0.107143,0.333333,0.120124,0.055238,0.109244,1.000000,0,-0.0952381,NaN,0.0857943,0.0857943,1
2,Kei Nishikori,Stanislas Wawrinka,1.0,-410.0,0.120715,-0.0690154,0.250000,0.0557803,-0.064673,0.086957,1.000000,-0.363636,-0.0454545,0.800000,-0.0206827,-0.0206827,1
3,Milos Raonic,Rafael Nadal,-6.0,2150.0,-0.0870152,0.167107,-0.151553,0.04946,-0.121952,0.071823,1.000000,0.655172,0.103448,0.357143,-0.0602382,-0.0602382,1
4,Grigor Dimitrov,Dominic Thiem,9.0,-1380.0,0.101972,0.0239085,0.014286,0.0451479,0.006865,0.068182,0.333333,-0.107143,-0.0714286,0.357143,0.0230751,0.0230751,1
5,Kei Nishikori,Jordan Thompson,-74.0,4216.0,-0.0369212,0.411654,0.403509,0.397997,0.027894,0.358025,1.000000,0.142857,-0.142857,NaN,0.160301,0.160301,1
9,Dominic Thiem,Samuel Groth,-172.0,3095.0,0.0734748,-0.153247,0.322981,0.0432361,-0.194828,0.073171,1.000000,0,0,0.250000,-0.137956,-0.137956,1
1000,Novak Djokovic,Nicolas Almagro,-74.0,6410.0,0.151564,-0.0893985,0.289803,0.0961202,0.042619,0.083799,1.000000,-0.137931,-0.0344828,0.025974,0.0607026,0.0607026,1
1001,Marius Copil,Guillermo Garcia Lopez,-20.0,70.0,0.116745,0.0277778,0.023109,0.0529186,-0.051798,0.017094,1.000000,0.0277778,-0.111111,-0.100000,-0.0200918,-0.0200918,1
1002,Borna Coric,Mischa Zverev,28.0,-344.0,-0.198871,0.0329268,0.489011,0.151674,-0.001830,0.134752,1.000000,0.181818,-0.136364,0.083333,0.0498485,0.0498485,1


In [66]:
results2 = results.filter(['RANK','FS','W1SP','W2SP','WSP','WRP','TPW','TMW','ACES','DF','BP','COMPLETE','SERVEADV','winner'], axis=1)

results2=results2.assign(player1=' ')
results2=results2.assign(player2=' ')

for i in range(0,len(results2.index)):
    if(results2.loc[i, 'RANK']<0):
        results2.loc[i, 'player1']=results.loc[i,'winner_name']
        results2.loc[i, 'player2']=results.loc[i,'loser_name']
    else:
        results2.loc[i, 'player1']=results.loc[i,'loser_name']
        results2.loc[i, 'player2']=results.loc[i,'winner_name']
        results2.loc[i, 'RANK':'SERVEADV']=results2.loc[i, 'RANK':'SERVEADV']*-1
        results2.loc[i, 'winner']=0

results2=results2.fillna(0)        
results2.loc[[0,1,2,3,4,5,9,1000,1001,1002,1003,1004]]

results2.to_csv('features2017.csv',header='false',index='false')
results2=pd.read_csv('features2017.csv')
results2.loc[[0,1,2,3,4,5,9,1000,1001,1002,1003,1004]]

,Unnamed: 0,RANK,FS,W1SP,W2SP,WSP,WRP,TPW,TMW,ACES,DF,BP,COMPLETE,SERVEADV,winner,player1,player2
0,0,-12.0,0.034820,-0.053768,-0.030000,-0.036138,-0.012796,-0.054795,-1.000000,-0.120000,-0.080000,-0.314286,-0.022834,-0.022834,0,Kei Nishikori,Grigor Dimitrov
1,1,-14.0,-0.161673,0.107143,-0.333333,-0.120124,-0.055238,-0.109244,-1.000000,-0.000000,0.095238,0.000000,-0.085794,-0.085794,0,Milos Raonic,Grigor Dimitrov
2,2,-1.0,-0.120715,0.069015,-0.250000,-0.055780,0.064673,-0.086957,-1.000000,0.363636,0.045455,-0.800000,0.020683,0.020683,0,Stanislas Wawrinka,Kei Nishikori
3,3,-6.0,-0.087015,0.167107,-0.151553,0.049460,-0.121952,0.071823,1.000000,0.655172,0.103448,0.357143,-0.060238,-0.060238,1,Milos Raonic,Rafael Nadal
4,4,-9.0,-0.101972,-0.023909,-0.014286,-0.045148,-0.006865,-0.068182,-0.333333,0.107143,0.071429,-0.357143,-0.023075,-0.023075,0,Dominic Thiem,Grigor Dimitrov
5,5,-74.0,-0.036921,0.411654,0.403509,0.397997,0.027894,0.358025,1.000000,0.142857,-0.142857,0.000000,0.160301,0.160301,1,Kei Nishikori,Jordan Thompson
9,9,-172.0,0.073475,-0.153247,0.322981,0.043236,-0.194828,0.073171,1.000000,0.000000,0.000000,0.250000,-0.137956,-0.137956,1,Dominic Thiem,Samuel Groth
1000,1000,-74.0,0.151564,-0.089398,0.289803,0.096120,0.042619,0.083799,1.000000,-0.137931,-0.034483,0.025974,0.060703,0.060703,1,Novak Djokovic,Nicolas Almagro
1001,1001,-20.0,0.116745,0.027778,0.023109,0.052919,-0.051798,0.017094,1.000000,0.027778,-0.111111,-0.100000,-0.020092,-0.020092,1,Marius Copil,Guillermo Garcia Lopez
1002,1002,-28.0,0.198871,-0.032927,-0.489011,-0.151674,0.001830,-0.134752,-1.000000,-0.181818,0.136364,-0.083333,-0.049848,-0.049848,0,Mischa Zverev,Borna Coric



### Soluciones Propuestas
Se propone evaluar si utilizar una red multicapa en el entorno tensorflow para la implementación del proyecto (el cual se entrena con todo el dataset) o una red neuronal recurrente la cual posee una memoria 'selectiva' de partidos anteriores. Estas soluciones están sujetas a cambiar de acuerdo a las nuevas técnicas a aprender en la materia.

### Medición de la Calidad de la Solución Obtenida
La solución obtenida se limitará a predecir el ganador de cada partido. Esto tiene como consecuencia determinar el resultado de cada uno de los torneos a nivel ATP de la temporada, como así también el resultado del ranking al finalizar la temporada. Se contrastaran con los resultados reales, pretendiéndose como mínimo una efectividad superior al 50%.

### Proyectos Similares Implementados en Internet
El proyecto de referencia será la tesis presente en el siguiente link: https://www.doc.ic.ac.uk/teaching/distinguished-projects/2015/m.sipko.pdf el cuál plantea la predicción de partidos de tenis con diversos métodos incluyendo Machine Learning y mostrando sus resultados. Otro proyecto más breve se encuentra en el siguiente link: http://deepakn94.github.io/assets/papers/6.867.pdf. La técnica para solucionar el problema también puede referirse a predictores de otros deportes.


In [58]:
X_train = results2.iloc[:, results2.columns.get_loc('FS'):results2.columns.get_loc('COMPLETE')].values
y_train = results2.iloc[:, results2.columns.get_loc('winner')].values

In [59]:
y_train

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

In [60]:
import keras
from keras.models import Sequential
from keras.layers import Dense

classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu', input_dim = X_train.shape[1]))

# Adding the second hidden layer
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))

# Compiling Neural Network
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])



c:\users\cufar\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=6, kernel_initializer="uniform", input_dim=10, activation="relu")`
  
c:\users\cufar\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=6, kernel_initializer="uniform", activation="relu")`
  # This is added back by InteractiveShellApp.init_path()
c:\users\cufar\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1, kernel_initializer="uniform", activation="sigmoid")`
  


In [61]:
# Fitting our model 
classifier.fit(X_train, y_train, batch_size = 10, nb_epoch = 100)

c:\users\cufar\anaconda3\envs\tensorflow\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/100
2534/2534 [==============================] - 0s 197us/step - loss: 0.5459 - acc: 0.8812
Epoch 2/100
2534/2534 [==============================] - 0s 117us/step - loss: 0.1073 - acc: 0.9882
Epoch 3/100
2534/2534 [==============================] - 0s 129us/step - loss: 0.0434 - acc: 0.9878
Epoch 4/100
2534/2534 [==============================] - 0s 123us/step - loss: 0.0346 - acc: 0.9882
Epoch 5/100
2534/2534 [==============================] - 0s 123us/step - loss: 0.0313 - acc: 0.9886
Epoch 6/100
2534/2534 [==============================] - 0s 129us/step - loss: 0.0297 - acc: 0.9886
Epoch 7/100
2534/2534 [==============================] - 0s 142us/step - loss: 0.0286 - acc: 0.9886
Epoch 8/100
2534/2534 [==============================] - 0s 117us/step - loss: 0.0278 - acc: 0.9890
Epoch 9/100
2534/2534 [==============================] - 0s 136us/step - loss: 0.0274 - acc: 0.9882
Epoch 10/100
2534/2534 [==============================] - 0s 148us/step - loss: 0.0270 - acc: 0.9882

In [62]:
# Predicting the Test set results
y_pred = classifier.predict(X_train)
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [ True],
       [False],
       [False]])

In [63]:
# Creating the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_train, y_pred)
cm

array([[ 882,   10],
       [  13, 1629]], dtype=int64)